Changes : 

* The field monitor worked a little bit, when looking at specific small angles.
* When looking wide for a good angle, results polluted by "good" candidates that trap light in the grating and reemit everywhere.
* As such, try with fiber again. But this time only emit from waveguide.

So go back to 2D_1
So in compute, remove  the look over sources and hardcode 1 = waveguide.

In [ ]:
from gc_outcoupler2 import initialize

In [ ]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument(
    "-period",
    type=float,
    default=0.767723445279288,
    help="grating period (default: 0.767723445279288 um)",
)
parser.add_argument(
    "-FF", type=float, default=0.5, help="Fill factor (default: 0.5 um)"
)
parser.add_argument(
    "-theta", type=float, default=8, help="fiber_angle (default: 8 degrees)"
)
parser.add_argument("-x", type=float, default=1, help="Fiber position (default: 1 um)")
parser.add_argument("-source", type=int, default=0, help="1 for waveguide, 0 for fiber")
parser.add_argument("-filename", type=str, default="default", help="data name")
args = parser.parse_args()
sim, fiber_monitor, waveguide_monitor = initialize(args)

In [ ]:
import matplotlib.pyplot as plt

f = plt.figure(dpi=100)
sim.plot2D(ax=f.gca())
plt.show()

In [ ]:
import meep as mp

f = plt.figure(dpi=100)
animate = mp.Animate2D(sim, mp.Ez, f=f, normalize=True)
sim.run(mp.at_every(1, animate), until=100)
plt.close()

In [ ]:
animate.to_jshtml(10)

In [ ]:
res_waveguide = sim.get_eigenmode_coefficients(
    waveguide_monitor, [1], eig_parity=mp.ODD_Z, direction=mp.X
)
kpoint = mp.Vector3(y=-1).rotate(mp.Vector3(z=1), -1 * 8)
res_fiber = sim.get_eigenmode_coefficients(
    fiber_monitor,
    [1],
    direction=mp.NO_DIRECTION,
    eig_parity=mp.ODD_Z,
    kpoint_func=lambda f, n: kpoint,
)

In [ ]:
res_waveguide.alpha

In [ ]:
res_fiber.alpha

In [ ]:
import numpy as np

np.abs(res_fiber.alpha[0, 0, 1]) ** 2 / np.abs(res_waveguide.alpha[0, 0, 0]) ** 2

In [ ]:
!mpirun -np 8 python gc_outcoupler2.py -period 0.767723445279288 -FF 0.5 -theta 8 -x 1 -source 1 -filename 'test' > ./logs/test.log

In [ ]:
from os import listdir
from os.path import isfile, join

onlyfiles = [f for f in listdir("./data") if isfile(join("./data", f))]

In [ ]:
onlyfiles[0]

In [ ]:
import pickle

dbfile = open("./data/" + onlyfiles[0], "rb")
db = pickle.load(dbfile)
dbfile.close()

In [ ]:
db

In [ ]:
import numpy as np

db["res_fiber"].alpha

In [ ]:
np.abs(db["res_fiber"].alpha[0, 0, 1]) ** 2 / np.abs(
    db["res_waveguide"].alpha[0, 0, 0]
) ** 2